# NSRR MESA Features extraction

https://sleepdata.org/datasets/mesa/pages/equipment/montage-and-sampling-rate-information.md

https://sleepdata.org/datasets/mesa/variables/overall5

In [ ]:
import os
import yasa
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
from preprocessing import crop_hypno, extract_features

# Define paths
root_dir = '/Volumes/NSRR/mesa/'
eeg_dir = root_dir + 'polysomnography/edfs/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/'
parent_dir = os.path.dirname(os.getcwd())
out_dir = parent_dir + '/output/features/'

# Keep training set of MESA
df_subj = pd.read_csv(parent_dir + "/output/demo/demo_nsrr_all.csv")
df_subj = df_subj.query("dataset == 'MESA' and set == 'training'").set_index("subj")

print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

In [ ]:
df = []
include = ['EEG3', 'EOG-L', 'EMG']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'mesa-sleep-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'mesa-sleep-' + str(sub) + '-profusion.xml'
    
    # Check that file exists
    if not os.path.isfile(eeg_file):
        warnings.warn("File not found %s" % eeg_file)
        continue
    if not os.path.isfile(hypno_file):
        warnings.warn("File not found %s" % hypno_file)
        continue

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
        raw.load_data()
    except:
        continue
        
    # Resample and low-pass filter 
    raw.resample(sf, npad="auto")

    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # (Optional) We keep up to 15 minutes before / after sleep
    # hypno, tmin, tmax = crop_hypno(hypno)
    # raw.crop(tmin, tmax)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA (DISABLED)
    # Hypnogram must include all stages
#     if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
#         print("- Not all stages are present.")
#         continue
#     # If the duration is not between 4 to 12 hours, skip subject
#     if not(4 < n_epochs / 120 < 12):
#         print("- Recording too short/long.")
#         continue

    # EXTRACT FEATURES
    features = extract_features(df_subj, sub, raw, include)
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [ ]:
# Add dataset
df['dataset'] = 'mesa'

# Convert to category
df['dataset'] = df['dataset'].astype('category')
df['stage'] = df['stage'].astype('category')

In [ ]:
# %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Median value of the EEG IQR per stage
df.groupby('stage')['eeg_iqr'].median()

In [ ]:
# Remove nights with a 9 in sleep stages
df.drop(index=df[df['stage'] == 9].index.get_level_values(0), level=0, inplace=True)
df['stage'] = df['stage'].astype(str).astype('category')
# Show %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Number of unique nights in dataset
df.index.get_level_values(0).nunique()

In [ ]:
# Export
df.to_parquet(out_dir + "features_nsrr_mesa.parquet")